### GET ACCESS TOKEN

In [4]:
import requests
import base64

CLIENT_SECRET='Your_client_secret'
CLIENT_ID='Your_client_id'

auth_string = f"{CLIENT_ID}:{CLIENT_SECRET}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

url = "https://accounts.spotify.com/api/token"

headers = {
    "Authorization": f"Basic {auth_base64}",
    "Content-Type": "application/x-www-form-urlencoded"
}

data = {
    "grant_type": "client_credentials"
}

response = requests.post(url, headers=headers, data=data)
token = response.json()["access_token"]

print(token)

BQDIpBJeJ3XntA79YMI-dos2qnKnakUL4Atei6r8z7pdWkghUCVZvKdV3VwCJdhuv0lLdxO3cAuMGkZK3c8qMQ78v81i8tCIvvNVAy95qBdh-Rc7267AgwbuUErhZY_6yGwsdJkcir4


In [3]:
print("Auth header:", headers["Authorization"])


Auth header: Basic OTMzNDZiODE4ODJkNGQwODhlYjc4OThmMGJkNDQxOWM6KCcyOGVhYzhkZTc1Y2U0ODk0YmZiZTAzZTZlMjQ2ZmI5OCcsKQ==


### GET DATA FROM SPOTIFY API

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from urllib.parse import urlparse, parse_qs
import time

# =========================
# AUTH SETUP
# =========================
sp_oauth = SpotifyOAuth(
    CLIENT_SECRET='Your_client_secret'
    CLIENT_ID='Your_client_id',
    redirect_uri="http://localhost:3000",
    scope="user-read-recently-played user-library-read playlist-read-private user-top-read",
    cache_path=".spotify_token.json"
)

token_info = sp_oauth.get_cached_token()

if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print("Go to this URL and log in:\n", auth_url)
    redirected = input("Paste FULL redirect URL here:\n")
    code = parse_qs(urlparse(redirected).query).get("code", [None])[0]
    token_info = sp_oauth.get_access_token(code)

sp = spotipy.Spotify(auth=token_info["access_token"])

# =========================
# ARTIST IMAGE CACHE
# =========================
artist_cache = {}

def get_artist_image(artist_id):
    if artist_id in artist_cache:
        return artist_cache[artist_id]

    try:
        artist = sp.artist(artist_id)
        img = artist["images"][0]["url"] if artist["images"] else None
        artist_cache[artist_id] = img
        return img
    except:
        return None

# =========================
# RECENTLY PLAYED
# =========================
def get_recently_played(limit=50):
    results = sp.current_user_recently_played(limit=limit)
    rows = []

    for item in results["items"]:
        t = item["track"]
        rows.append({
            "track_name": t["name"],
            "artist": t["artists"][0]["name"],
            "album": t["album"]["name"],
            "played_at": item["played_at"],
            "track_id": t["id"],
            "song_link": t["external_urls"]["spotify"],
            "album_link": t["album"]["external_urls"]["spotify"],
            "album_cover": t["album"]["images"][0]["url"] if t["album"]["images"] else None,
            "artist_link": t["artists"][0]["external_urls"]["spotify"],
            "artist_image": get_artist_image(t["artists"][0]["id"])
        })

    return pd.DataFrame(rows)

# =========================
# SAVED TRACKS
# =========================
def get_saved_tracks():
    rows, offset = [], 0

    while True:
        results = sp.current_user_saved_tracks(limit=50, offset=offset)

        for item in results["items"]:
            t = item["track"]
            rows.append({
                "track_name": t["name"],
                "artist": t["artists"][0]["name"],
                "album": t["album"]["name"],
                "added_at": item["added_at"],
                "track_id": t["id"],
                "song_link": t["external_urls"]["spotify"],
                "album_link": t["album"]["external_urls"]["spotify"],
                "album_cover": t["album"]["images"][0]["url"] if t["album"]["images"] else None,
                "artist_link": t["artists"][0]["external_urls"]["spotify"],
                "artist_image": get_artist_image(t["artists"][0]["id"])
            })

        if results["next"] is None:
            break

        offset += 50
        time.sleep(0.1)

    return pd.DataFrame(rows)

# =========================
# PLAYLISTS + SONGS
# =========================
def get_playlists_with_songs():
    playlists = sp.current_user_playlists()["items"]
    rows = []

    for p in playlists:
        playlist_id = p["id"]
        playlist_name = p["name"]
        offset = 0

        while True:
            items = sp.playlist_items(playlist_id, offset=offset, limit=100)

            for item in items["items"]:
                if not item["track"]:
                    continue
                t = item["track"]

                rows.append({
                    "playlist": playlist_name,
                    "track_name": t["name"],
                    "artist": t["artists"][0]["name"],
                    "album": t["album"]["name"],
                    "track_id": t["id"],
                    "song_link": t["external_urls"]["spotify"],
                    "album_link": t["album"]["external_urls"]["spotify"],
                    "album_cover": t["album"]["images"][0]["url"] if t["album"]["images"] else None,
                    "artist_link": t["artists"][0]["external_urls"]["spotify"],
                    "artist_image": get_artist_image(t["artists"][0]["id"])
                })

            if items["next"] is None:
                break

            offset += 100
            time.sleep(0.1)

    return pd.DataFrame(rows)

# =========================
# TOP TRACKS / ARTISTS
# =========================
def get_top_items(item_type, term):
    items = sp.current_user_top_tracks(limit=50, time_range=term) if item_type == "tracks" \
        else sp.current_user_top_artists(limit=50, time_range=term)

    rows = []

    for t in items["items"]:
        if item_type == "tracks":
            rows.append({
                "track_name": t["name"],
                "artist": t["artists"][0]["name"],
                "track_id": t["id"],
                "time_range": term,
                "song_link": t["external_urls"]["spotify"],
                "album": t["album"]["name"],
                "album_link": t["album"]["external_urls"]["spotify"],
                "album_cover": t["album"]["images"][0]["url"] if t["album"]["images"] else None,
                "artist_link": t["artists"][0]["external_urls"]["spotify"],
                "artist_image": get_artist_image(t["artists"][0]["id"])
            })
        else:
            rows.append({
                "artist": t["name"],
                "genres": t["genres"],
                "popularity": t["popularity"],
                "artist_link": t["external_urls"]["spotify"],
                "artist_image": get_artist_image(t["id"]),
                "time_range": term
            })

    return pd.DataFrame(rows)

# =========================
# RUN EXTRACTION
# =========================
print("Pulling Spotify data...")

df_recent = get_recently_played()
df_saved = get_saved_tracks()
df_playlists = get_playlists_with_songs()
df_top_short = get_top_items("tracks", "short_term")
df_top_medium = get_top_items("tracks", "medium_term")
df_top_long = get_top_items("tracks", "long_term")
df_top_artists = get_top_items("artists", "long_term")

# =========================
# EXPORT
# =========================
output_path = r"C:\Users\EMMAN\Downloads\Spotify_Data.xlsx"

with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    df_recent.to_excel(writer, sheet_name="Recently Played", index=False)
    df_saved.to_excel(writer, sheet_name="Saved Tracks", index=False)
    df_playlists.to_excel(writer, sheet_name="Playlists", index=False)
    df_top_short.to_excel(writer, sheet_name="Top Tracks (Short)", index=False)
    df_top_medium.to_excel(writer, sheet_name="Top Tracks (Medium)", index=False)
    df_top_long.to_excel(writer, sheet_name="Top Tracks (Long)", index=False)
    df_top_artists.to_excel(writer, sheet_name="Top Artists", index=False)

print("✅ Spotify_Data.xlsx created successfully")


Pulling Spotify data...
✅ Spotify_Data.xlsx created successfully


### GET ADDITIONAL LINKS

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.cache_handler import MemoryCacheHandler
from tqdm import tqdm
import os

# --- CONFIGURATION ---
CLIENT_SECRET='Your_client_secret'
CLIENT_ID='Your_client_id'
INPUT_EXCEL = r"C:\Users\EMMAN\Downloads\Streaming_History_Audio_2023-2025_Streaming_History.csv" 
OUTPUT_FILE = r"C:\Users\EMMAN\Downloads\Spotify_Enriched_Workbook.xlsx"

# Initialize Spotify API
auth_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, 
    client_secret=CLIENT_SECRET,
    cache_handler=MemoryCacheHandler()
)
sp = spotipy.Spotify(auth_manager=auth_manager)

def get_spotify_data(series, search_type):
    """Generic function to search Spotify for unique values in a Series."""
    # Drop empty rows and get unique names to save API calls
    unique_items = series.dropna().unique()
    results = []
    
    print(f"Processing {len(unique_items)} unique {search_type}s...")
    
    for item_name in tqdm(unique_items):
        try:
            # Search Spotify
            res = sp.search(q=str(item_name), type=search_type, limit=1)
            
            if search_type == 'track' and res['tracks']['items']:
                data = res['tracks']['items'][0]
                results.append({
                    'Original Name': item_name,
                    'Spotify Link': data['external_urls']['spotify'],
                    'Image Link': data['album']['images'][0]['url'] if data['album']['images'] else None
                })
            
            elif search_type == 'album' and res['albums']['items']:
                data = res['albums']['items'][0]
                results.append({
                    'Original Name': item_name,
                    'Spotify Link': data['external_urls']['spotify'],
                    'Image Link': data['images'][0]['url'] if data['images'] else None
                })
                
            elif search_type == 'artist' and res['artists']['items']:
                data = res['artists']['items'][0]
                results.append({
                    'Original Name': item_name,
                    'Spotify Link': data['external_urls']['spotify'],
                    'Image Link': data['images'][0]['url'] if data['images'] else None
                })
            else:
                results.append({'Original Name': item_name, 'Spotify Link': 'NOT FOUND', 'Image Link': None})
        except Exception as e:
            print(f"Error processing {item_name}: {e}")
            results.append({'Original Name': item_name, 'Spotify Link': 'ERROR', 'Image Link': None})
            
    return pd.DataFrame(results)

def main():
    if not os.path.exists(INPUT_EXCEL):
        print(f"Error: Could not find the file at {INPUT_EXCEL}.")
        return

    # 1. Read the CSV file with automatic encoding detection
    print("Reading CSV file...")
    encodings_to_try = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252', 'utf-16']
    
    df = None
    for encoding in encodings_to_try:
        try:
            df = pd.read_csv(INPUT_EXCEL, encoding=encoding)
            print(f"Successfully read file using {encoding} encoding")
            break
        except (UnicodeDecodeError, UnicodeError):
            continue
    
    if df is None:
        print("Error: Could not read file with any common encoding")
        return
    
    # 2. Print available columns to help debug
    print("\nAvailable columns:")
    print(df.columns.tolist())
    print()

    # 3. Verify columns exist before processing
    # Common Spotify history column names (adjust based on your actual columns)
    possible_track_cols = ['master_metadata_track_name', 'Master Metadata Track Name', 'track_name']
    possible_album_cols = ['master_metadata_album_album_name', 'Master Metadata Album Album Name', 'album_name']
    possible_artist_cols = ['master_metadata_album_artist_name', 'Master Metadata Album Artist Name', 'artist_name']
    
    # Find the actual column names
    track_col = None
    album_col = None
    artist_col = None
    
    for col in df.columns:
        if col.lower() in [c.lower() for c in possible_track_cols]:
            track_col = col
        elif col.lower() in [c.lower() for c in possible_album_cols]:
            album_col = col
        elif col.lower() in [c.lower() for c in possible_artist_cols]:
            artist_col = col
    
    if not track_col or not album_col or not artist_col:
        print("Error: Could not find required columns.")
        print(f"Looking for track column (found: {track_col})")
        print(f"Looking for album column (found: {album_col})")
        print(f"Looking for artist column (found: {artist_col})")
        return

    # 4. Process each column specifically
    tracks_df = get_spotify_data(df[track_col], 'track')
    albums_df = get_spotify_data(df[album_col], 'album')
    artists_df = get_spotify_data(df[artist_col], 'artist')

    # 5. Save to a single Excel Workbook with three sheets
    print(f"\nSaving enriched data to {OUTPUT_FILE}...")
    with pd.ExcelWriter(OUTPUT_FILE, engine='xlsxwriter') as writer:
        tracks_df.to_excel(writer, sheet_name='Tracks', index=False)
        albums_df.to_excel(writer, sheet_name='Albums', index=False)
        artists_df.to_excel(writer, sheet_name='Artists', index=False)

    print(f"\nSuccess! File saved at: {os.path.abspath(OUTPUT_FILE)}")

if __name__ == "__main__":
    main()

Reading CSV file...
Successfully read file using utf-8 encoding

Available columns:
['Ts', 'Platform', 'Ms Played', 'Conn Country', 'Ip Addr', 'Master Metadata Track Name', 'Master Metadata Album Artist Name', 'Master Metadata Album Album Name', 'Spotify Track Uri', 'Episode Name', 'Episode Show Name', 'Spotify Episode Uri', 'Audiobook Title', 'Audiobook Uri', 'Audiobook Chapter Uri', 'Audiobook Chapter Title', 'Reason Start', 'Reason End', 'Shuffle', 'Skipped', 'Offline', 'Offline Timestamp', 'Incognito Mode']

Processing 1129 unique tracks...


 87%|████████████████████████████████████████████████████████████████████▋          | 981/1129 [07:05<00:59,  2.49it/s]

### DOWNLOAD THE IMAGES

In [1]:
import pandas as pd
import requests
import os
import re
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import time

# --- CONFIGURATION ---
INPUT_WORKBOOK = r"C:\Users\EMMAN\Downloads\Spotify_Enriched_Workbook.xlsx"
FOLDERS = {
    'Artist': r"C:\Users\EMMAN\OneDrive\Documents\My Tableau Repository\Shapes\Artist Image",
    'Album': r"C:\Users\EMMAN\OneDrive\Documents\My Tableau Repository\Shapes\Album covers",
    'Track': r"C:\Users\EMMAN\OneDrive\Documents\My Tableau Repository\Shapes\Tracks covers"
}

for folder in FOLDERS.values():
    os.makedirs(folder, exist_ok=True)

def sanitize_filename(filename):
    """Removes characters that are illegal in Windows filenames."""
    if not isinstance(filename, str) or filename.lower() == 'nan':
        return "Unknown"
    clean = re.sub(r'[<>:"/\\|?*]', '', filename).strip()
    return clean[:150]

def download_and_resize_image(url, folder, filename, size=(640, 640), max_retries=3):
    """Downloads, resizes to specific px, converts to RGB, and saves as .jpg with retries"""
    if not url or pd.isna(url) or str(url).lower() in ['none', 'not found', 'error']:
        return False, "No valid URL"
    
    filepath = os.path.join(folder, f"{filename}.jpg")
    
    # Double check if file exists
    if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
        return True, "Already exists"
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=15, headers={'User-Agent': 'Mozilla/5.0'})
            
            if response.status_code == 200 and len(response.content) > 0:
                # Load image from the response content
                img = Image.open(BytesIO(response.content))
                
                # Convert to RGB (required for saving as JPEG)
                if img.mode != "RGB":
                    img = img.convert("RGB")
                
                # Resize using Lanczos filter for high quality
                img_resized = img.resize(size, Image.Resampling.LANCZOS)
                
                # Save as JPEG with high quality
                img_resized.save(filepath, "JPEG", quality=95, optimize=True)
                
                # Verify file was actually saved
                if os.path.exists(filepath) and os.path.getsize(filepath) > 0:
                    return True, "Downloaded successfully"
                else:
                    return False, "File not saved properly"
            else:
                if attempt < max_retries - 1:
                    time.sleep(1)  # Wait before retry
                    continue
                return False, f"HTTP {response.status_code}"
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(1)  # Wait before retry
                continue
            return False, str(e)
    
    return False, "Max retries exceeded"

def download_missing_only():
    """Only download items that are missing from folders"""
    
    for sheet_name, folder_key in [('Artists', 'Artist'), ('Albums', 'Album'), ('Tracks', 'Track')]:
        print(f"\n{'='*60}")
        print(f"PROCESSING {sheet_name.upper()}")
        print('='*60)
        
        df = pd.read_excel(INPUT_WORKBOOK, sheet_name=sheet_name)
        folder = FOLDERS[folder_key]
        
        # Get list of actual files in folder
        actual_files = set(f.replace('.jpg', '') for f in os.listdir(folder) if f.endswith('.jpg'))
        
        # Find missing items
        missing = []
        for _, row in df.iterrows():
            original_name = str(row['Original Name'])
            sanitized_name = sanitize_filename(original_name)
            
            if sanitized_name not in actual_files:
                url = row['Image Link']
                has_url = not (pd.isna(url) or str(url).lower() in ['none', 'not found', 'error', 'nan'])
                if has_url:  # Only add if it has a valid URL
                    missing.append({
                        'name': sanitized_name,
                        'url': url
                    })
        
        if not missing:
            print(f"✓ All images present! ({len(actual_files)} files)")
            continue
        
        print(f"Found {len(missing)} missing items with valid URLs. Downloading...")
        
        success_count = 0
        failed_count = 0
        
        for item in tqdm(missing, desc=f"Downloading {folder_key}"):
            success, reason = download_and_resize_image(item['url'], folder, item['name'])
            if success:
                success_count += 1
            else:
                failed_count += 1
                print(f"\n  ❌ Failed: {item['name']} - {reason}")
        
        print(f"\n✓ Successfully downloaded: {success_count}")
        print(f"❌ Failed: {failed_count}")
        
        # Final count
        final_files = len([f for f in os.listdir(folder) if f.endswith('.jpg')])
        print(f"Total .jpg files now: {final_files}")

if __name__ == "__main__":
    download_missing_only()


PROCESSING ARTISTS
Found 298 missing items with valid URLs. Downloading...



✓ Successfully downloaded: 298
❌ Failed: 0
Total .jpg files now: 297

PROCESSING ALBUMS
Found 806 missing items with valid URLs. Downloading...



✓ Successfully downloaded: 806
❌ Failed: 0
Total .jpg files now: 802

PROCESSING TRACKS
Found 1130 missing items with valid URLs. Downloading...



✓ Successfully downloaded: 1130
❌ Failed: 0
Total .jpg files now: 1118


### KNOW WHICH IMAGES ARE MISSING

In [1]:
import pandas as pd
import os
import re

INPUT_WORKBOOK = r"C:\Users\EMMAN\Downloads\Spotify_Enriched_Workbook.xlsx"
FOLDERS = {
    'Artist': r"C:\Users\EMMAN\OneDrive\Documents\My Tableau Repository\Shapes\Artist Image",
    'Album': r"C:\Users\EMMAN\OneDrive\Documents\My Tableau Repository\Shapes\Album covers",
    'Track': r"C:\Users\EMMAN\OneDrive\Documents\My Tableau Repository\Shapes\Tracks covers"
}

def sanitize_filename(filename):
    """Removes characters that are illegal in Windows filenames."""
    if not isinstance(filename, str) or filename.lower() == 'nan':
        return "Unknown"
    clean = re.sub(r'[<>:"/\\|?*]', '', filename).strip()
    return clean[:150]

def find_missing_images():
    for sheet_name, folder_key in [('Artists', 'Artist'), ('Albums', 'Album'), ('Tracks', 'Track')]:
        print(f"\n{'='*60}")
        print(f"CHECKING {sheet_name.upper()}")
        print('='*60)
        
        df = pd.read_excel(INPUT_WORKBOOK, sheet_name=sheet_name)
        folder = FOLDERS[folder_key]
        
        # Get list of actual files in folder
        actual_files = set(f.replace('.jpg', '') for f in os.listdir(folder) if f.endswith('.jpg'))
        
        missing = []
        for _, row in df.iterrows():
            original_name = str(row['Original Name'])
            sanitized_name = sanitize_filename(original_name)
            
            if sanitized_name not in actual_files:
                url = row['Image Link']
                has_url = not (pd.isna(url) or str(url).lower() in ['none', 'not found', 'error', 'nan'])
                missing.append({
                    'Original': original_name,
                    'Sanitized': sanitized_name,
                    'Has URL': has_url,
                    'URL': url if has_url else 'No URL'
                })
        
        if missing:
            print(f"\n❌ {len(missing)} items missing from folder:")
            for item in missing:
                status = "✓ Has URL" if item['Has URL'] else "❌ No URL"
                print(f"\n   Original: '{item['Original']}'")
                print(f"   Sanitized: '{item['Sanitized']}'")
                print(f"   {status}: {item['URL']}")
        else:
            print(f"✓ All {len(df)} images present in folder")
        
        print(f"\nExpected: {len(df)} | Found: {len(actual_files)} | Missing: {len(missing)}")

if __name__ == "__main__":
    find_missing_images()


CHECKING ARTISTS

❌ 3 items missing from folder:

   Original: 'Iyanu'
   Sanitized: 'Iyanu'
   ❌ No URL: No URL

   Original: 'The Gospel Messages'
   Sanitized: 'The Gospel Messages'
   ❌ No URL: No URL

   Original: 'Bbo'
   Sanitized: 'Bbo'
   ✓ Has URL: https://i.scdn.co/image/ab6761610000e5eb27bb0b12d0e8a42e50a13c34

Expected: 300 | Found: 297 | Missing: 3

CHECKING ALBUMS

❌ 4 items missing from folder:

   Original: 'You Are Great'
   Sanitized: 'You Are Great'
   ✓ Has URL: https://i.scdn.co/image/ab67616d0000b273588a126005750b198d90ebdf

   Original: 'Alignment Chant'
   Sanitized: 'Alignment Chant'
   ✓ Has URL: https://i.scdn.co/image/ab67616d0000b273f823f8e0f81c2e20d84d65f5

   Original: 'ELOHIM ADONAI'
   Sanitized: 'ELOHIM ADONAI'
   ✓ Has URL: https://i.scdn.co/image/ab67616d0000b2736a4bec5b9efbf3fd519a7f69

   Original: 'Fathered By The Best'
   Sanitized: 'Fathered By The Best'
   ✓ Has URL: https://i.scdn.co/image/ab67616d0000b273fc7b0e1b426a3a615b556705

Expected: 

### CONVERT JSON TO CSV

In [17]:
import json
import pandas as pd

json_file = r"C:\Users\EMMAN\Downloads\my_spotify_data\Spotify Extended Streaming History\Streaming_History_Audio_2023-2026.json"
output_csv = r"C:\Users\EMMAN\Downloads\Streaming_History_Audio_2023-2026.csv"

df = pd.read_json(json_file)

# Convert timestamp
if 'ts' in df.columns:
    df['ts'] = pd.to_datetime(df['ts'])

# Clean column names
df.columns = df.columns.str.replace("_", " ").str.title()

df.to_csv(output_csv, index=False, encoding="utf-8-sig")

print("Done!")

Done!
